# Alabama - Analysis

## Import the Requireed Packages

In [2]:
# For working with graphs
import networkx as nx
# For making plots
import matplotlib.pyplot as plt
import numpy as np

# Needed for gerrychain
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import geopandas as gpd
import pandas as pd

# Other necessary packages
import random
import maup

## Initial Plan - Struck down by Courts

In [3]:
# Read shapefile data into a geo data frame and a graph
al_init_gdf = gpd.read_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_201/2021 Alabama Congressional Plan_shape file.shp")
al_init_graph = Graph.from_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_201/2021 Alabama Congressional Plan_shape file.shp", ignore_errors=True)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column LOCKED!
  warnings.warn("NA values found in column {}!".format(column))
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column NAME!
  warnings.warn("NA values found in column {}!".format(column))


In [6]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
al_init_gdf

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,F18_POP,F18_WHT,F18_BLK,F18_IND,F18_ASN,F18_HWN,F18_OTH,IDEAL_VALU,DEVIATION,F_DEVIATIO,F_WHITE,F_BLACK,F_18_POP,F_18_WHT,F_18_BLK,F_18_IND,F_18_ASN,F_18_HWN,F_18_OTH,DISTRICT_L,geometry
0,2,6612.89502,1,1.0,None,CD1,717754,461324,186921,557535,371902,138128,6381,8395,290,7947,717754.0,0.0,0.000000,0.642733,0.260425,0.776777,0.667047,0.247748,0.011445,0.015057,0.000520,0.014254,1|0%,"POLYGON ((-87.59883 30.99745, -87.59884 30.997..."
1,3,10532.34270,2,1.0,None,CD2,717755,433244,217392,557677,350279,162714,2628,10399,307,9802,717754.0,1.0,0.000001,0.603610,0.302878,0.776974,0.628104,0.291771,0.004712,0.018647,0.000550,0.017576,2|0%,"POLYGON ((-85.91929 32.27438, -85.91237 32.274..."
2,4,8460.39062,3,1.0,None,CD3,717754,479432,176953,564281,386048,136382,2048,9869,340,8505,717754.0,0.0,0.000000,0.667961,0.246537,0.786176,0.684141,0.241692,0.003629,0.017490,0.000603,0.015072,3|0%,"POLYGON ((-86.55384 33.01893, -86.55355 33.018..."
3,5,9058.35644,4,1.0,None,CD4,717754,582698,51929,556133,463433,39834,5475,3427,245,18651,717754.0,0.0,0.000000,0.811835,0.072349,0.774824,0.833313,0.071627,0.009845,0.006162,0.000441,0.033537,4|0%,"POLYGON ((-86.14981 34.53363, -86.14864 34.533..."
4,6,3502.37963,5,1.0,None,CD5,717754,499707,124642,561187,403155,95757,4130,10814,447,15080,717754.0,0.0,0.000000,0.696209,0.173656,0.781865,0.718397,0.170633,0.007359,0.019270,0.000797,0.026872,5|0%,"MULTIPOLYGON (((-86.14981 34.53363, -86.14981 ..."
5,7,3261.12426,6,1.0,None,CD6,717754,498843,138019,552286,397498,100878,2183,10568,254,16611,717754.0,0.0,0.000000,0.695006,0.192293,0.769464,0.719732,0.182655,0.003953,0.019135,0.000460,0.030077,6|0%,"POLYGON ((-87.18652 33.55695, -87.18712 33.557..."
6,8,11021.27630,7,1.0,None,None,717754,265204,400306,568067,222731,308030,1707,7036,232,10629,717754.0,0.0,0.000000,0.369491,0.557720,0.791451,0.392086,0.542242,0.003005,0.012386,0.000408,0.018711,7|0%,"POLYGON ((-87.26692 33.51293, -87.26718 33.513..."


We need to confirm that there aren't any issues in the Districts, so we can use the "maup" package.

In [8]:
# Check geodataframe using maup's doctor function
maup.doctor(al_init_gdf)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True

## The Second Plan

In [9]:
# Read shapefile data into a geo data frame and a graph
al_adopt_gdf = gpd.read_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_adopted_2023/SP_Remedial_Plan_3 2023-10-05.shp")
al_adopt_graph = Graph.from_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_adopted_2023/SP_Remedial_Plan_3 2023-10-05.shp", ignore_errors=True)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column LOCKED!
  warnings.warn("NA values found in column {}!".format(column))
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column NAME!
  warnings.warn("NA values found in column {}!".format(column))


In [10]:
al_adopt_gdf

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,AP_WHT,AP_BLK,F18_POP,F18_WHT,F18_BLK,F18_AP_WHT,F18_AP_BLK,IDEAL_VALU,DEVIATION,F_DEVIATIO,F_WHITE,F_BLACK,F_AP_WHT,F_AP_BLK,F_18_POP,F_18_WHT,F_18_BLK,F_18_AP_WH,F_18_AP_BL,DISTRICT_L,geometry
0,1,8126.240234,1,1.0,None,None,717754,527330,116462,566761,126801,557336,421681,86271,446497,90582,717754.0,0.0,0.000000,0.734695,0.162259,0.789631,0.176664,0.776500,0.756601,0.154792,0.801127,0.162527,1|0%,"POLYGON ((-88.19693 31.14404, -88.19317 31.144..."
1,2,9562.906250,2,1.0,None,None,717754,303461,353228,328547,364985,558661,251320,265638,267394,272023,717754.0,0.0,0.000000,0.422792,0.492130,0.457743,0.508510,0.778346,0.449861,0.475491,0.478634,0.486920,2|0%,"POLYGON ((-85.88615 32.49305, -85.88575 32.493..."
2,3,6873.106934,3,1.0,None,None,717754,508080,146376,539477,156204,564595,408820,112550,429007,116843,717754.0,0.0,0.000000,0.707875,0.203936,0.751618,0.217629,0.786614,0.724094,0.199346,0.759849,0.206950,3|0%,"POLYGON ((-86.57780 33.76532, -86.57780 33.766..."
3,4,8855.813477,4,1.0,None,None,717754,585183,49721,621226,56723,555217,465284,37403,488583,40112,717754.0,0.0,0.000000,0.815297,0.069273,0.865514,0.079028,0.773548,0.838022,0.067366,0.879986,0.072246,4|0%,"POLYGON ((-88.27462 33.53401, -88.27459 33.534..."
4,5,4067.180420,5,1.0,None,None,717754,495427,126226,543486,138438,560406,399106,97134,429649,102735,717754.0,0.0,0.000000,0.690246,0.175862,0.757204,0.192877,0.780777,0.712173,0.173328,0.766675,0.183322,5|0%,"MULTIPOLYGON (((-86.14981 34.53363, -86.14981 ..."
5,6,4556.976074,6,1.0,None,None,717755,517634,125785,552665,133963,552411,409699,93329,431902,96958,717754.0,1.0,0.000001,0.721185,0.175248,0.769991,0.186642,0.769637,0.741656,0.168948,0.781849,0.175518,6|0%,"POLYGON ((-87.03982 33.68746, -87.03982 33.687..."
6,7,10406.541992,7,1.0,None,None,717754,283337,378364,306688,387622,568540,239136,289398,255566,295119,717754.0,0.0,0.000000,0.394755,0.527150,0.427288,0.540049,0.792110,0.420614,0.509020,0.449513,0.519082,7|0%,"POLYGON ((-87.26692 33.51293, -87.26718 33.513..."


We need to confirm that there aren't any issues in the Districts, so we can use the "maup" package.

In [11]:
# Check geodataframe using maup's doctor function
maup.doctor(al_adopt_gdf)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True

## Join the Census Precinct Data to the Districting Plans

In [ ]:
# Import 2022 General Election voting data
al_init_data_gdf = gpd.read_file('sc_2022_gen_prec/sc_2022_gen_st_prec/sc_2022_gen_st_prec.shp')
al_init_data_gdf.columns